In [ ]:
import folium
import math
import json
from geopy.distance import distance

# -----------------------------
# Configuration and Parameters
# -----------------------------

# Trier, Germany (approximate center)
center_lat = 49.75
center_lon = 6.63
center_point = (center_lat, center_lon)

# Destination point (for drawing lines and calculating air distance)
dest_point = (49.64061136958772, 6.272924336004428)

# Hexagon settings
hex_size = 0.3      # side length of each hexagon in kilometers (100 m)
grid_radius = 90     # maximum hex "distance" in axial coordinates (yields 91 hexagons)

# -----------------------------
# Helper Functions
# -----------------------------

def axial_to_offset(q, r, size):
    """
    For a pointy-topped hexagon grid using axial coordinates,
    convert (q, r) into an (east, north) offset in kilometers.
      - Horizontal offset: size * sqrt(3) * (q + r/2)
      - Vertical offset: size * 1.5 * r
    """
    x = size * math.sqrt(3) * (q + r / 2)
    y = size * 1.5 * r
    return (x, y)

def offset_to_latlon(origin, offset_x, offset_y):
    """
    Given an origin (lat, lon) and an offset (in km) in the east (x) and north (y)
    directions, return the destination (lat, lon) using geopy.
    """
    d = math.sqrt(offset_x**2 + offset_y**2)
    if d == 0:
        return origin
    # Calculate bearing in degrees (0° = north)
    bearing = math.degrees(math.atan2(offset_x, offset_y))
    destination = distance(kilometers=d).destination(origin, bearing)
    return (destination.latitude, destination.longitude)

def hex_distance(q, r):
    """
    Compute the "hex distance" (number of steps) from (0,0) to (q, r)
    using cube coordinate conversion.
    """
    return int((abs(q) + abs(r) + abs(-q - r)) / 2)

def polygon_vertices(center, size):
    """
    Given a hexagon center (lat, lon) and side length (in km), compute the 6 vertices
    of a pointy-topped hexagon. The vertices are at angles: 60*i - 30 for i=0,...,5.
    """
    vertices = []
    for i in range(6):
        angle_deg = 60 * i - 30
        angle_rad = math.radians(angle_deg)
        offset_x = size * math.cos(angle_rad)
        offset_y = size * math.sin(angle_rad)
        vertex = offset_to_latlon(center, offset_x, offset_y)
        vertices.append(vertex)
    return vertices

# -----------------------------
# Create the Map and Add Elements
# -----------------------------

# Create a Folium map centered on Trier (with a high zoom level to see 100 m details)
m = folium.Map(location=[center_lat, center_lon], zoom_start=18)

# Add a marker for Trier center
folium.Marker(
    location=center_point,
    popup="Trier Center",
    icon=folium.Icon(icon="home", color="blue")
).add_to(m)

# Add a marker for the destination point
folium.Marker(
    location=dest_point,
    popup="Destination",
    icon=folium.Icon(icon="flag", color="red")
).add_to(m)

# List to hold hexagon center information (to later populate the sidebar and JavaScript array)
hex_points = []

# Loop over axial coordinates (q, r) and add hexagons and center points
for q in range(-grid_radius, grid_radius + 1):
    for r in range(-grid_radius, grid_radius + 1):
        if hex_distance(q, r) <= grid_radius:
            # Compute hexagon center relative to Trier
            offset_x, offset_y = axial_to_offset(q, r, hex_size)
            hex_center = offset_to_latlon(center_point, offset_x, offset_y)
            
            # Compute the vertices for the hexagon polygon (each vertex is 100 m from center)
            verts = polygon_vertices(hex_center, hex_size)
            
            # Add the hexagon polygon (blue outline with light fill)
            folium.Polygon(
                locations=verts,
                color='blue',
                fill=True,
                fill_opacity=0.2,
                weight=1,
                popup=f"Hex ({q}, {r})"
            ).add_to(m)
            
            # Calculate the air (great-circle) distance from this hexagon center to the destination
            air_dist = distance(hex_center, dest_point).km
            
            # Save hexagon center info
            hex_points.append({
                "q": q,
                "r": r,
                "lat": hex_center[0],
                "lon": hex_center[1],
                "air_distance": air_dist
            })
            
            # Add a small dot for the hexagon center using a CircleMarker
            folium.CircleMarker(
                location=hex_center,
                radius=2,       # small dot
                color='green',
                fill=True,
                fill_color='green'
            ).add_to(m)

# -----------------------------
# (Optional) Build a Sidebar Listing All Hexagon Centers
# -----------------------------

sidebar_html = '''
<div id="sidebar" style="
    position: absolute;
    top: 10px;
    left: 10px;
    width: 250px;
    height: 90%;
    background-color: white;
    overflow: auto;
    z-index: 9999;
    padding: 10px;
    border: 2px solid gray;">
  <h4>Hexagon Centers</h4>
  <ul style="list-style-type: none; padding-left: 5px;">
'''
for pt in hex_points:
    sidebar_html += f'''
    <li style="margin-bottom: 5px;">
      Hex ({pt['q']}, {pt['r']}) - {pt['air_distance']:.2f} km
    </li>
    '''
sidebar_html += '''
  </ul>
</div>
'''
m.get_root().html.add_child(folium.Element(sidebar_html))

# -----------------------------
# Add Custom JavaScript for Drawing Lines Automatically
# -----------------------------

# Convert hex_points list to JSON so it can be used in JavaScript
hex_points_json = json.dumps(hex_points)

# Get the map’s JavaScript variable name (e.g. "map_xxxx")
map_name = m.get_name()

# Custom JavaScript: This code automatically draws a red line from each hexagon center to the destination
# when the page loads. (No popups or alerts appear.)
custom_js = f'''
<script>
  // Destination point (lat, lon)
  var destination = [{dest_point[0]}, {dest_point[1]}];
  var drawnLines = [];
  
  // Draw a red line from (lat, lon) to the destination.
  function drawLineFromPoint(lat, lon) {{
    var line = L.polyline([[lat, lon], destination], {{color: 'red'}}).addTo({map_name});
    drawnLines.push(line);
  }}

  // Draw lines from every hexagon center to the destination.
  function drawAllLines() {{
    hexPoints.forEach(function(pt) {{
      drawLineFromPoint(pt.lat, pt.lon);
    }});
  }}

  // Array of all hexagon centers (populated from Python)
  var hexPoints = {hex_points_json};

  // Once the document is fully loaded, draw all lines automatically.
  document.addEventListener("DOMContentLoaded", function() {{
    drawAllLines();
  }});
</script>
'''

m.get_root().html.add_child(folium.Element(custom_js))

# -----------------------------
# Save the Map
# -----------------------------

m.save("trier_hexgrid_map_all_lines.html")
print("Map saved to trier_hexgrid_map_all_lines.html")


In [1]:
import folium
import math
import requests
from geopy.distance import distance

# -----------------------------
# Configuration and Parameters
# -----------------------------

# Trier, Germany (approximate center)
center_lat = 49.75
center_lon = 6.63
center_point = (center_lat, center_lon)

# Work destination (to evaluate driving time)
dest_point = (49.64061136958772, 6.272924336004428)

# Hexagon settings
hex_size = 0.3      # side length in kilometers (100 m)
grid_radius = 80     # grid radius in axial steps (yields 91 hexagons)

# -----------------------------
# Helper Functions
# -----------------------------

def axial_to_offset(q, r, size):
    """
    Convert axial coordinates (q, r) to (east, north) offset in kilometers.
    For pointy-topped hexagons:
      - Horizontal offset = size * sqrt(3) * (q + r/2)
      - Vertical offset = size * 1.5 * r
    """
    x = size * math.sqrt(3) * (q + r / 2)
    y = size * 1.5 * r
    return (x, y)

def offset_to_latlon(origin, offset_x, offset_y):
    """
    Given an origin (lat, lon) and an offset in km (east, north),
    compute the destination lat/lon using geopy.
    """
    d = math.sqrt(offset_x**2 + offset_y**2)
    if d == 0:
        return origin
    bearing = math.degrees(math.atan2(offset_x, offset_y))  # bearing in degrees, 0° = north
    destination = distance(kilometers=d).destination(origin, bearing)
    return (destination.latitude, destination.longitude)

def hex_distance(q, r):
    """
    Compute hex grid "distance" from (0,0) using cube coordinate conversion.
    """
    return int((abs(q) + abs(r) + abs(-q - r)) / 2)

def polygon_vertices(center, size):
    """
    Compute the 6 vertices of a pointy-topped hexagon given its center (lat, lon)
    and side length (in km). Vertices are computed at angles: 60*i - 30.
    """
    vertices = []
    for i in range(6):
        angle_deg = 60 * i - 30
        angle_rad = math.radians(angle_deg)
        offset_x = size * math.cos(angle_rad)
        offset_y = size * math.sin(angle_rad)
        vertex = offset_to_latlon(center, offset_x, offset_y)
        vertices.append(vertex)
    return vertices

def get_driving_time(source_lat, source_lon, dest_lat, dest_lon):
    """
    Use the OSRM API to get the driving time (in minutes) from source to destination.
    OSRM expects coordinates in "lon,lat" order.
    """
    # Prepare coordinates in "lon,lat" format
    source_str = f"{source_lon},{source_lat}"
    dest_str = f"{dest_lon},{dest_lat}"
    url = f"http://router.project-osrm.org/route/v1/driving/{source_str};{dest_str}?overview=false"
    try:
        response = requests.get(url)
        data = response.json()
        if "routes" in data and len(data["routes"]) > 0:
            duration_seconds = data["routes"][0]["duration"]
            return duration_seconds / 60.0  # convert to minutes
    except Exception as e:
        print("OSRM request failed:", e)
    return None

# -----------------------------
# Create the Map and Add Elements
# -----------------------------

m = folium.Map(location=[center_lat, center_lon], zoom_start=18)

# Add marker for Trier center and work destination
folium.Marker(location=center_point, popup="Trier Center", icon=folium.Icon(icon="home", color="blue")).add_to(m)
folium.Marker(location=dest_point, popup="Work Destination", icon=folium.Icon(icon="flag", color="red")).add_to(m)

# Loop over axial coordinates to generate hexagons
for q in range(-grid_radius, grid_radius + 1):
    for r in range(-grid_radius, grid_radius + 1):
        if hex_distance(q, r) <= grid_radius:
            # Compute the hexagon center
            offset_x, offset_y = axial_to_offset(q, r, hex_size)
            hex_center = offset_to_latlon(center_point, offset_x, offset_y)
            verts = polygon_vertices(hex_center, hex_size)

            # Query OSRM API to get driving time from hex_center to work destination
            driving_time = get_driving_time(hex_center[0], hex_center[1], dest_point[0], dest_point[1])
            
            # Determine color based on driving time (if available)
            if driving_time is not None:
                # Example thresholds: under 15 min = green; 15-30 = orange; over 30 = red.
                if driving_time <= 15:
                    color = "green"
                elif driving_time <= 30:
                    color = "orange"
                else:
                    color = "red"
                popup_text = f"Driving time: {driving_time:.1f} min"
            else:
                color = "gray"
                popup_text = "Driving time: N/A"
            
            # Add hexagon polygon with the chosen color
            folium.Polygon(
                locations=verts,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.4,
                weight=1,
                popup=f"Hex ({q}, {r}): {popup_text}"
            ).add_to(m)
            
            # Optionally add a small center marker
            folium.CircleMarker(
                location=hex_center,
                radius=2,
                color="black",
                fill=True,
                fill_color="black"
            ).add_to(m)

# -----------------------------
# Save the Map
# -----------------------------

m.save("hexagon_driving_time_map.html")
print("Map saved to hexagon_driving_time_map.html")


KeyboardInterrupt: 

In [2]:
import folium
import math
import requests
from geopy.distance import distance

# -----------------------------
# Konfiguration
# -----------------------------
# Trier (Zentrum)
center_lat = 49.64
center_lon = 6.27
center_point = (center_lat, center_lon)

# Arbeitsziel (zum Beispiel)
dest_point = (49.64061136958772, 6.272924336004428)

# Hexagon-Einstellungen
hex_size = 1.0      # Seitenlänge in Kilometern (1 km)
grid_radius = 10    # Axialer Grid-Radius (viele Hexagone)

# -----------------------------
# Hilfsfunktionen
# -----------------------------
def axial_to_offset(q, r, size):
    """
    Berechnet den Offset (in km) eines Hexagon-Zentrums im axialen Koordinatensystem.
    """
    x = size * math.sqrt(3) * (q + r / 2)
    y = size * 1.5 * r
    return (x, y)

def offset_to_latlon(origin, offset_x, offset_y):
    """
    Wandelt einen Offset (in km) von einem Ursprungspunkt (lat, lon) in einen Zielpunkt (lat, lon) um.
    """
    d = math.sqrt(offset_x**2 + offset_y**2)
    if d == 0:
        return origin
    bearing = math.degrees(math.atan2(offset_x, offset_y))  # 0° = Norden
    destination = distance(kilometers=d).destination(origin, bearing)
    return (destination.latitude, destination.longitude)

def hex_distance(q, r):
    """
    Berechnet die "Hex-Distanz" von (0,0) zu (q, r) (Anzahl Schritte im Hex-Gitter).
    """
    return int((abs(q) + abs(r) + abs(-q - r)) / 2)

def polygon_vertices(center, size):
    """
    Berechnet die 6 Eckpunkte eines pointy-topped Hexagons anhand des Zentrums und der Seitenlänge (in km).
    """
    vertices = []
    for i in range(6):
        angle_deg = 60 * i - 30
        angle_rad = math.radians(angle_deg)
        offset_x = size * math.cos(angle_rad)
        offset_y = size * math.sin(angle_rad)
        vertex = offset_to_latlon(center, offset_x, offset_y)
        vertices.append(vertex)
    return vertices

def get_driving_time(source_lat, source_lon, dest_lat, dest_lon):
    """
    Fragt über OSRM die Fahrzeit (in Minuten) von source zu dest ab.
    OSRM erwartet die Koordinaten im Format "lon,lat".
    """
    source_str = f"{source_lon},{source_lat}"
    dest_str = f"{dest_lon},{dest_lat}"
    url = f"http://router.project-osrm.org/route/v1/driving/{source_str};{dest_str}?overview=false"
    try:
        response = requests.get(url)
        data = response.json()
        if "routes" in data and len(data["routes"]) > 0:
            duration_seconds = data["routes"][0]["duration"]
            return duration_seconds / 60.0  # Minuten
    except Exception as e:
        print("OSRM-Anfrage fehlgeschlagen:", e)
    return None

def get_color(driving_time):
    """
    Ermittelt eine Farbe (als Hex-String) für einen bestimmten Fahrzeitwert (in Minuten)
    im Bereich von 5 bis 70 Minuten. Bei <= 5 min: grün (#00ff00), bei >= 70 min: dunkelrot (#8b0000).
    Dazwischen erfolgt eine lineare Interpolation.
    """
    if driving_time is None:
        return "gray"
    lower_bound = 5.0
    upper_bound = 70.0
    # Clamp
    if driving_time < lower_bound:
        driving_time = lower_bound
    if driving_time > upper_bound:
        driving_time = upper_bound
    ratio = (driving_time - lower_bound) / (upper_bound - lower_bound)
    # Interpolation: von grün (0,255,0) zu dunkelrot (139,0,0)
    r = round(ratio * 139)            # Rotanteil steigt von 0 bis 139
    g = round((1 - ratio) * 255)        # Grünanteil fällt von 255 bis 0
    b = 0
    return f"#{r:02x}{g:02x}{b:02x}"

# -----------------------------
# Karte erstellen und Hexagone hinzufügen
# -----------------------------
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Marker für Trier und Arbeitsziel
folium.Marker(location=center_point, popup="Trier", icon=folium.Icon(icon="home", color="blue")).add_to(m)
folium.Marker(location=dest_point, popup="Arbeitsziel", icon=folium.Icon(icon="flag", color="red")).add_to(m)

# Schleife über das Hexagon-Gitter
for q in range(-grid_radius, grid_radius + 1):
    for r in range(-grid_radius, grid_radius + 1):
        if hex_distance(q, r) <= grid_radius:
            # Hexagon-Zentrum berechnen
            offset_x, offset_y = axial_to_offset(q, r, hex_size)
            hex_center = offset_to_latlon(center_point, offset_x, offset_y)
            verts = polygon_vertices(hex_center, hex_size)
            
            # Fahrzeit von diesem Zentrum zum Arbeitsziel ermitteln
            driving_time = get_driving_time(hex_center[0], hex_center[1], dest_point[0], dest_point[1])
            color = get_color(driving_time)
            
            # Hexagon zeichnen – Farbe entspricht dem Verlauf
            folium.Polygon(
                locations=verts,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.4,
                weight=1,
                popup=f"Fahrzeit: {driving_time:.1f} Min" if driving_time is not None else "Fahrzeit: N/A"
            ).add_to(m)

            # Optional: Kleiner Mittelpunkt als Punkt
            folium.CircleMarker(
                location=hex_center,
                radius=2,
                color="black",
                fill=True,
                fill_color="black"
            ).add_to(m)

# -----------------------------
# Karte speichern
# -----------------------------
m.save("hexagon_driving_time_gradient.html")
print("Karte gespeichert als hexagon_driving_time_gradient.html")


OSRM-Anfrage fehlgeschlagen: Expecting value: line 1 column 1 (char 0)
OSRM-Anfrage fehlgeschlagen: Expecting value: line 1 column 1 (char 0)
OSRM-Anfrage fehlgeschlagen: Expecting value: line 1 column 1 (char 0)
OSRM-Anfrage fehlgeschlagen: Expecting value: line 1 column 1 (char 0)
OSRM-Anfrage fehlgeschlagen: Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 